# LTV

LTV (lifetime value) - прибыль, которую в среднем принесёт пользователь за время взаимодействия с сервисом.
Расчёт LTV зависит от модели, которая описывает поведение пользователей.

## Модель для сервисов работающих по подписке

Например, для сервисов с подпиской эта модель может быть такой. Пользователь оформляет подписку с величиной платежа $F$ и каждый месяц либо продолжает ей пользоваться, либо отменяет с вероятностью $p$ (эта вероятность называется оттоком или коэффициентом оттока). Тогда (c учётом ряда допущений) LTV можно вычислить как произведение среднего платёжа на среднее время подписки - $\frac{1}{p} $ :
$$ LTV = \frac{{average\_payment}}{p} $$

Проверим, как ведёт себя эта оценка на практике, сгенерировав синтетические данные о дате и величине платежа.

In [1]:
import numpy as np
import pandas as pd
import datetime
import hashlib

np.random.seed(0)

# число пользователей
N = 10000 

# величина платежа для разных уровней подписки
grades = np.array([10, 20, 50, 100]) 

# отток
p = 0.05 

payments = grades[np.random.randint(4, size=(N,))] #случайным образом назначаем пользователям уровни подписки
months = np.random.geometric(p, size=(N,))         # и длительность взаимодействия в месяцах


# сгенерируем псевдослучайные e-mail для идентификации пользователей в данных
def get_email(i):
    hash_ = hashlib.sha256(str(i).encode('utf-8')).hexdigest()[:10]
    return f"{hash_}@example.com"

def get_payment_dates(i, months_with_sub):
    def next_month(date):
        try:
            return datetime.date(date.year, date.month+1, date.day)
        except ValueError:
            return datetime.date(date.year+1, 1, date.day)
    date = datetime.date(2020, 10, i%20+1)
    dates = [date]
    for j in range(months_with_sub-1):
        date = next_month(date)
        dates.append(date)
    return dates

data = []
for i, payment, months_with_sub in zip(range(N), payments, months):
    data.extend(list(zip([get_email(i)]*months_with_sub, get_payment_dates(i, months_with_sub), [payment] * months_with_sub)))

dataframe = pd.DataFrame(data, columns=['email', 'payment_date', 'payment_value'])

Получаем следующую таблицу.

In [2]:
dataframe

,email,payment_date,payment_value
0,5feceb66ff@example.com,2020-10-01,10
1,5feceb66ff@example.com,2020-11-01,10
2,5feceb66ff@example.com,2020-12-01,10
3,5feceb66ff@example.com,2021-01-01,10
4,5feceb66ff@example.com,2021-02-01,10
...,...,...,...
195439,9daf56fbee@example.com,2021-06-19,100
195440,9daf56fbee@example.com,2021-07-19,100
195441,888df25ae3@example.com,2020-10-20,10
195442,888df25ae3@example.com,2020-11-20,10


Вычислим LTV (по состоянию на февраль) и сравним с реальным.

In [3]:
# средний платёж
mean_payment_estimate = dataframe[dataframe['payment_date'] <= datetime.date(2021, 1, 31)]['payment_value'].mean() 
mean_payment_real = grades.mean()

#количество пользователей в декабре и январе
december_users = dataframe[(datetime.date(2020, 12, 1) <= dataframe['payment_date']) & (dataframe['payment_date'] <= datetime.date(2020, 12, 31))]['email'].nunique()
january_users = dataframe[(datetime.date(2021, 1, 1) <= dataframe['payment_date']) & (dataframe['payment_date'] <= datetime.date(2021, 1, 31))]['email'].nunique()

# отток
p_estimate = (december_users - january_users)/december_users 

# LTV оценка
LTV_estimated = mean_payment_estimate / p_estimate

# LTV 
LTV_real = mean_payment_real / p

print(f"Реальный LTV: {LTV_real}, оценка: {LTV_estimated}.")

Реальный LTV: 900.0, оценка: 867.3646054241912.


Рассмотрим ещё одну модель взаимодействия пользователя с сервисом.

## Модель для сервисов с единовременными платежами

В этой модели пользователь совершает несколько платежей с произвольным интервалом между ними и затем прекращает взаимодействие с сервисом. Тогда LTV аналогично предыдущей модели можно вычислить как произведение среднего платежа на среднее количество платежей от пользователя:

$$ LTV = average\_payment \cdot average\_payments $$

Однако, в отличие от предыдущей модели, хорошую оценку можно получить в этом случае только собрав данные за период равный или больший среднего времени взаимодействия пользователя с сервисом. Интуитивно это следует из того, что в предыдущей модели пользователи сообщают дополнительную информацию самим фактом отписки. Оценим LTV на тех же данных исходя из новой модели. На этот раз получим оценки для нескольких моментов во времени и соответствующего количества собранных данных.

In [4]:
four_months_data = dataframe[dataframe['payment_date'] <= datetime.date(2021, 1, 31)]
one_year_data = dataframe[dataframe['payment_date'] <= datetime.date(2021, 9, 30)]
three_years_data = dataframe[dataframe['payment_date'] <= datetime.date(2023, 10, 31)]

# средний платёж
mean_payment_estimate_four_months_data = four_months_data['payment_value'].mean() 
mean_payment_estimate_one_year_data    = one_year_data['payment_value'].mean() 
mean_payment_estimate_three_years_data = three_years_data['payment_value'].mean() 

#среднее количество платежей
mean_payments_estimate_four_months_data = four_months_data.groupby('email').count()['payment_date'].mean()
mean_payments_estimate_one_year_data = one_year_data.groupby('email').count()['payment_date'].mean()
mean_payments_estimate_three_years_data = three_years_data.groupby('email').count()['payment_date'].mean()

# LTV
LTV_four_months_data_estimate = mean_payment_estimate_four_months_data * mean_payments_estimate_four_months_data
LTV_one_year_data_estimate = mean_payment_estimate_one_year_data * mean_payments_estimate_one_year_data
LTV_three_years_data_estimate = mean_payment_estimate_three_years_data * mean_payments_estimate_three_years_data

print(
f"""Реальный LTV: {LTV_real}
Оценка (4 месяца данных): {LTV_four_months_data_estimate}
Оценка (1 год данных): {LTV_one_year_data_estimate}
Оценка (3 года данных): {LTV_three_years_data_estimate}
""")


Реальный LTV: 900.0
Оценка (4 месяца данных): 165.78900000000002
Оценка (1 год данных): 410.035
Оценка (3 года данных): 755.536



## Заключение
Общий подход к оценке LTV и вообще любых показателей можно описать такой схемой:
- выбор и описание модели (параметры модели и связи между ними, вероятностные или детерминированные, априорные распределения параметров)
- получение апостериорных распределений параметров
- вычисление необходимых показателей с помощью точечных оценок или методом Монте-Карло